This a brief tutorial to use HDP-GPC, a dynamic clustering library. 
As a summary of the objectives of this notebook:
1. Learn to load some public data and learn how compute the initial statistics.
2. Learn to define the model and check each one of the hyperparameters' configuration.
3. Learn to execute inference and how this can be tuned.
4. Learn how to plot the results and study the dynamic parameters.

We should first import the main packages to deal with the data:

In [ ]:
import numpy as np
import torch
import os

Now we can load the data, to illustrate this example we are going to reduce the data to examples from 1700 to 1950.

In [ ]:
cwd = os.path.dirname(os.getcwd())
data_path = os.path.join(cwd, 'data', 'mitbih')
data = np.load(os.path.join(data_path, '100.npy'))
labels = np.load(os.path.join(data_path, '100_labels.npy'))

print("Shape of the data: ")
print(data.shape)
data = data[1700:1950]
labels = labels[1700:1950]
num_samples, num_obs_per_sample, num_outputs = data.shape

The data dimensions correspond with [number of examples, number of observations, number of outputs]. Let's check how looks this data:

In [ ]:
import matplotlib.pyplot as plt

plt.plot(data[0,:,0])
plt.show()

As we can see, it corresponds to a heartbeat. This is the kind of patterns we want to cluster over the record.
To compute the initial statistics we are going to use the observation variance and the 1-step rolled variance.
For that, we have a method in the get_data class, where we can choose the batch to compute this statistics.

In [ ]:
from hdpgpc.get_data import compute_estimators_LDS

std, std_dif = compute_estimators_LDS(data, n_f=50)
print("Variance: ", str(std))
print("1-step variance: ", str(std_dif))

As we can see, the 1-step variance is higher than the variance, which indicates a slight dynamic behaviour.

Now we can define the HDP-GPC model. 

The hyperparameters can be directly estimated from the previously computed estimators. As a standard an initial M has to be chosen, as a default, it is chosen as M=2 as the model computes the needed number of clusters as it progresses in the inference.


In [ ]:
M = 2
sigma = [std * 1.0] * M
bound_sigma_ = (std * 0.1, std * 0.2)
gamma = [std_dif * 1.0] * M
bound_gamma = (std_dif * 0.1, std_dif * 1.0)

We can also define the kernel parameters, in this case we are taking outputscale = 300.0 as is the mean amplitude found on a standard record of ECG. Also, we are choosing ini_legthscale = 1.0 as a standard value. Both can be tuned, but they do not have a critical impact on the inference because they are optimized in terms of Maximum-Likelihood.

In [ ]:
outputscale_ = 300.0
ini_lengthscale = 3.0
bound_lengthscale = (1.0, 20.0)

Also, we have to define the time indexes where the observations had been taken, which is x_train. In this case it can be as simple as a range of the same length of the observations, but it can differ along the samples. On the other hand, we have to define x_basis, which is the time index basis where the LDS is going to be computed (it fix the dimension of the LDS) If desired, the x_basis can be reduced to be a set of inducing points. If this dimensional reduction is applied it should be reflected in the option parameter inducing_points=True. 

In [ ]:
samples = [0, num_obs_per_sample]
l, L = samples[0], samples[1]
x_basis = np.atleast_2d(np.arange(l, L, 1, dtype=np.float64)).T
x_train = np.atleast_2d(np.arange(l, L, dtype=np.float64)).T
#If x_basis is wanted to be smaller than the observations length, then the inducing points approach can be applied setting this parameter to True.
inducing_points = False

Additionally, we can also define the warp GP hyperparameters and time indexes. 

In [ ]:
#Choose if warp is going to be applied. (In the most recent version is optimized to work with online inference, but it can be used in offline as an additional step at the end of the clustering).
warp = False
#Warp priors
noise_warp = std * 0.1
bound_noise_warp = (noise_warp * 0.1, noise_warp * 0.2)
#Warp time indexes
x_basis_warp = np.atleast_2d(np.arange(l, L, 2, dtype=np.float64)).T

Finally, with these parameters we can define the HDP-GPC. 
There exist a lot of configuration parameters that can be combined in this model, to check the options and the relevance of these configuration options go to the Documentation (in progress).

In terms of define the model, we should consider the inference method we want to apply because each one have their restrictions (by now).

1. In first place we have the online inference, where we can include one sample at a time and let the model cluster it. By now, the online inference only works with one output signal, but it is going to be extended soon. That's why we choose n_outputs=1.

In [ ]:
import hdpgpc.GPI_HDP as hdpgp

sw_gp = hdpgp.GPI_HDP(x_basis=x_basis, x_basis_warp=x_basis_warp, n_outputs=1,
                          ini_lengthscale=ini_lengthscale, bound_lengthscale=bound_lengthscale,
                          ini_gamma=gamma, ini_sigma=sigma, ini_outputscale=outputscale_, noise_warp=noise_warp,
                          bound_sigma=bound_sigma_, bound_gamma=bound_gamma, bound_noise_warp=bound_noise_warp,
                          verbose=True, max_models=100, inducing_points=inducing_points, estimation_limit=30)

We can revise the priors that had been defined for our model.  The cluster list of GP-LDS is generated inside the model, indexes as gpmodels[n_output][n_cluster]. We can plot the zero-knowledge cluster.

In [ ]:
sw_gp.gpmodels[0][0].plot_last(0)

In [ ]:
data_ = data[:,:,[0]]
sw_gp.include_sample(x_train, data_[0], with_warp=warp)

Now the first example has been included in the model. 
We can check some parameters to be sure the sample has been added:

In [ ]:
sw_gp.gpmodels[0][0].plot_last(0)

As we can see, the sample had been added successfully and the GP performed regression over it. Now we can include more samples, in this case, we are adding 100 to check how fast is the computation without warp component.

In [ ]:
import time

for j, d in enumerate(data_[1:200]):
    start_time = time.time()
    print("Sample:", j+1, "/", str(data_.shape[0]-1), "label:", labels[j+1])
    sw_gp.include_sample(x_train, d, with_warp=warp)
    print("Time --- %s seconds ---" % (time.time() - start_time))

The clustering has generated two clusters. Let's revise them:

In [ ]:
sw_gp.gpmodels[0][0].plot_last(0)
sw_gp.gpmodels[0][1].plot_last(0)

Even if we want to revise the beats included we can use the util_plot methods. Here the difference between groups is clearer:

In [ ]:
from hdpgpc.util_plots import print_results, plot_models_plotly

main_model = print_results(sw_gp, labels, 0, error=False)
selected_gpmodels = sw_gp.selected_gpmodels()
plot_models_plotly(sw_gp, selected_gpmodels, main_model, labels, N_0=0, lead=0, step=0.5, plot_latent=True)

If we try to include the examples from 1900 to 1950, knowing that a ventricular beat exist among them, we are going to check if the model generates a new group.

In [ ]:
plt.plot(data_[200:,:,0].T)
plt.show()


In [ ]:

for j, d in enumerate(data_[200:]):
    start_time = time.time()
    print("Sample:", j + 200, "/", str(data_.shape[0]-1), "label:", labels[j+200])
    sw_gp.include_sample(x_train, d, with_warp=warp)
    print("Time --- %s seconds ---" % (time.time() - start_time))

In [ ]:
main_model = print_results(sw_gp, labels, 0, error=False)
selected_gpmodels = sw_gp.selected_gpmodels()
plot_models_plotly(sw_gp, selected_gpmodels, main_model, labels, N_0=0, lead=0, step=0.5, plot_latent=True)

As expected, the ventricular beat generated a cluster itself.

To perform the same clustering but in an offline scheme, we only have to choose the offline inference method, we are going to use it with full record. Here are some new option parameters: reestimate_initial_params indicates when a reestimation for the prior LDS is desired (it is recommended in the offline setting), n_explore_steps indicates the number of tryals to generate a new group on each iteration, recommended is above 10, but in this case to make the clustering faster, 3 will be enough. This full clustering takes a bit longer.

In [ ]:
sw_gp = hdpgp.GPI_HDP(x_basis=x_basis, x_basis_warp=x_basis_warp, n_outputs=1,
                          ini_lengthscale=ini_lengthscale, bound_lengthscale=bound_lengthscale,
                          ini_gamma=gamma, ini_sigma=sigma, ini_outputscale=outputscale_, noise_warp=noise_warp,
                          bound_sigma=bound_sigma_, bound_gamma=bound_gamma, bound_noise_warp=bound_noise_warp,
                          verbose=True, max_models=100, inducing_points=inducing_points, reestimate_initial_params=True,
                          n_explore_steps=3)

data = np.load(os.path.join(data_path, '100.npy'))[:,:,[0]]
labels = np.load(os.path.join(data_path, '100_labels.npy'))
num_samples = data.shape[0]
x_trains = np.array([x_train] * num_samples)
sw_gp.include_batch(x_trains, data)


Now we can plot the results same as before:

In [ ]:
main_model = print_results(sw_gp, labels, 0, error=False)
selected_gpmodels = sw_gp.selected_gpmodels()
plot_models_plotly(sw_gp, selected_gpmodels, main_model, labels, N_0=0, lead=0, step=0.5, plot_latent=True)

Of course, depending on the nature of the record to cluster, the hyperparameters can be tuned. First consideration should be focused on determining if the record is benefited of a dynamic clustering or not. In the case the patterns exhibit a dynamic change while remaining in the same category it will be interesting to priorize a dynamic behaviour, which is determined making $S_{\omega} > S_{\epsilon}$. If there exist different morphologies with patterns really close one to another, then it will be better to prioritize a static behaviour, with the opposite condition $S_{\omega} < S_{\epsilon}$. 

Once we have the clustering performed, there are parameters of the model which can lead us to interesting insights. For example, the transition matrix represent the probability of jumping from one cluster to another, which can be useful to detect some prevalent loops of behaviour.

In [ ]:
sw_gp.compute_Pi()[:-1,:-1]


Also, visualizing the dynamic parameters can be useful in some cases:

In [ ]:
plt.imshow(sw_gp.gpmodels[0][0].Gamma[-1])
plt.colorbar()
plt.show()
plt.imshow(sw_gp.gpmodels[0][0].Sigma[-1])
plt.colorbar()
plt.show()

Usually, the dynamic transformation matrices $A, C$ are really close to the identity due to the conditions they have to satisfy (they have to be stable). So, sometimes is better to represent the transformation they produce in a vector of ones.

In [ ]:
plt.plot(torch.matmul(sw_gp.gpmodels[0][0].A[-1], torch.ones(x_basis.shape[0])))
plt.plot(torch.matmul(sw_gp.gpmodels[0][1].A[-1], torch.ones(x_basis.shape[0])))
plt.plot()

Here we can see how even if the normality clusters seems really similar, the underlying dynamic transformation is really different. Also, we can use the Kullback-Leibler divergence to measure the similarity of the models during their evolution and then represent this distances using a MDS reduction.

In [ ]:
from hdpgpc.util_plots import plot_MDS
plot_MDS(sw_gp, main_model, labels, N_0=0, save=None)